# Update the extraction results

In [1]:
import os
import datetime
import json
from pathlib import Path

import numpy as np
import pandas as pd
from pydantic import BaseModel
from amplifai import Amplifier
from langchain_openai import ChatOpenAI
from langchain_mistralai import ChatMistralAI
from dotenv import load_dotenv
from baker.schemas.ingredient import Ingredient
from baker.schemas.recipe import Recipe, ParsedRecipe
from baker.schemas.step import Step

In [2]:
load_dotenv()

True

In [3]:
cwd = os.getcwd()
path_to_data = Path(cwd).parent / "data"
path_to_input = path_to_data / "input"
path_to_output = path_to_data / "output"
path_to_recipes_v1 = path_to_input / "recipes_v1.json"

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MISTRALAI_API_KEY = os.getenv("MISTRAL_API_KEY")

In [5]:
df_recipes_v1 = pd.read_json(path_to_recipes_v1)
df_recipes_v1.head()

,title,date,tags,introduction,ingredients,direction,output
0,Creamy Mashed Potatoes,2021-03-12,"[potato, side, cheesefare]",![Creamy Mashed Potatoes](/pix/creamy-mashed-p...,The quantities here are for about four adult p...,1. Peel and cut the potatoes into medium sized...,"{'title': 'Creamy Mashed Potatoes', 'ingredien..."
1,Red Sauce (Ragu all'Italiana),2021-03-16,"[italian, sauce]",My great-grandma's red sauce. All purpose: goo...,- 1/3 lb salt pork - 2 lb chuck roast - 3 Clov...,"1. Mince the garlic, peel the carrot, peel and...","{'title': 'Red Sauce (Ragu all'Italiana)', 'in..."
2,Turmeric Flatbread,2022-08-09,"[bread, turkish]",A great companion to Turkish Red Lentil Soup. ...,- 1 cup Wheat Flour (white or whole) - 1/2 Tbs...,"1. Combine flour, turmeric, salt and baking po...","{'title': 'Turmeric Flatbread', 'ingredients':..."
3,Zurich-Style Meat Saute,2023-01-06,"[beef, cream, swiss, quick]","Originally called ""Züri Gschnätzlets"" (Zurich ...","- 600g\tBeef, finely sliced - 250g\tMushrooms,...",1. Fry the meat in butter at high heat until i...,"{'title': 'Zurich-Style Meat Saute', 'ingredie..."
4,Tuscan Style Pork Roast,2021-03-10,"[italian, pork, roast]",![tuscan-style-pork-roast](/pix/tuscan-style-p...,- 1 pork Roast - 2-3 Tbsp fresh rosemary - 8 c...,1.\tPreheat oven to 275°F (135°C) 2.\tButterfl...,"{'title': 'Tuscan Style Pork Roast', 'ingredie..."


In [14]:
openai_llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0.7, model="gpt-4o")
mistral_llm = ChatMistralAI(
    api_key=MISTRALAI_API_KEY, temperature=0.7, model="mistral-large-latest"
)

In [15]:
human_prompt = """Given the following recipe introduction, ingredients and steps. Please structure the recipe as a NoSQL document.

Introduction:
{introduction}

Ingredients:
{ingredients}

Steps:
{steps}

"""

In [16]:
openai_amplifier = Amplifier[ChatOpenAI, ParsedRecipe](
    llm=openai_llm,
    human_prompt=human_prompt,
)

mistral_amplifier = Amplifier[ChatMistralAI, ParsedRecipe](
    llm=mistral_llm,
    human_prompt=human_prompt,
)

In [17]:
recipes_v1 = df_recipes_v1.to_dict(orient="records")

In [18]:
first_recipe = recipes_v1[0]

In [19]:
parsed_recipe = openai_amplifier.denoise(
    introduction=first_recipe["introduction"],
    ingredients=first_recipe["ingredients"],
    steps=first_recipe["direction"],
)
parsed_recipe

ParsedRecipe(serving_size=4, preparation_time=None, cooking_time=None, ingredients=[Ingredient(id=1, name='potatoes', quantity=1.0, unit='kilogram', optional=False), Ingredient(id=2, name='milk', quantity=200.0, unit='ml', optional=False), Ingredient(id=3, name='mayonnaise', quantity=200.0, unit='ml', optional=False), Ingredient(id=4, name='cheese', quantity=100.0, unit='gram', optional=False), Ingredient(id=5, name='Garlic powder', quantity=None, unit='N/A', optional=False), Ingredient(id=6, name='bacon', quantity=Range(min=12.0, max=16.0), unit='strip', optional=False), Ingredient(id=7, name='Butter', quantity=None, unit='N/A', optional=False), Ingredient(id=8, name='green onions', quantity=Range(min=3.0, max=4.0), unit='unit', optional=False), Ingredient(id=9, name='Black pepper', quantity=None, unit='N/A', optional=False), Ingredient(id=10, name='Salt', quantity=None, unit='N/A', optional=False)], steps=[Step(number=1, description='Peel and cut the potatoes into medium sized pieces

In [20]:
parsed_recipe.model_dump()

{'serving_size': 4,
 'preparation_time': None,
 'cooking_time': None,
 'ingredients': [{'id': 1,
   'name': 'potatoes',
   'quantity': 1.0,
   'unit': 'kilogram',
   'optional': False},
  {'id': 2,
   'name': 'milk',
   'quantity': 200.0,
   'unit': 'ml',
   'optional': False},
  {'id': 3,
   'name': 'mayonnaise',
   'quantity': 200.0,
   'unit': 'ml',
   'optional': False},
  {'id': 4,
   'name': 'cheese',
   'quantity': 100.0,
   'unit': 'gram',
   'optional': False},
  {'id': 5,
   'name': 'Garlic powder',
   'quantity': None,
   'unit': 'N/A',
   'optional': False},
  {'id': 6,
   'name': 'bacon',
   'quantity': {'min': 12.0, 'max': 16.0},
   'unit': 'strip',
   'optional': False},
  {'id': 7,
   'name': 'Butter',
   'quantity': None,
   'unit': 'N/A',
   'optional': False},
  {'id': 8,
   'name': 'green onions',
   'quantity': {'min': 3.0, 'max': 4.0},
   'unit': 'unit',
   'optional': False},
  {'id': 9,
   'name': 'Black pepper',
   'quantity': None,
   'unit': 'N/A',
   'option

In [13]:
mistral_parsed_recipe = mistral_amplifier.denoise(
    introduction=first_recipe["introduction"],
    ingredients=first_recipe["ingredients"],
    steps=first_recipe["direction"],
)
mistral_parsed_recipe.model_dump()

ValidationError: 1 validation error for ParsedRecipe
serving_size
  Field required [type=missing, input_value={'ingredients': [{'id': 1...hopped green onions.'}]}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing